### Imports, classes, and functions

In [1]:
import json
import requests
import os

class PokeURL:
    def __init__(self, __base_url: str = 'https://pokeapi.co/api/v2/', endpoint='', **kwargs):
        self.params = '&'.join([f'{k}={v}' for k,v in kwargs.items()])
        self.url_with_endpoint = f'{__base_url}{endpoint}/'
        self.url = f'{self.url_with_endpoint}?{self.params}/' if self.params else self.url_with_endpoint

    def __eq__(self, other):
        return f'{self.url}' == f'{other}'
    
    def __str__(self):
        return f'{self.url}'
    
def get_pokemon_response(url: str) -> dict:
    response = requests.get(url)
    response.raise_for_status()
    return response.json()

### Tests

In [46]:
test_url = PokeURL(endpoint='digimon', id=1, test='test', smells_like='teen_spirit')
assert test_url == 'https://pokeapi.co/api/v2/digimon/?id=1&test=test&smells_like=teen_spirit/'

True


### Getting all Pokemon

In [39]:
def get_all_pokemon() -> list:
    pokemon = []

    MAX_LIMIT = 60
    url = PokeURL(endpoint='pokemon', limit=MAX_LIMIT)
    data = get_pokemon_response(url)

    results = data.get('results')
    pokemon.extend(results)
    next_url = data.get('next')

    while next_url:
        data = get_pokemon_response(next_url)
        
        results = data.get('results')
        pokemon.extend(results)
        next_url = data.get('next')

    return pokemon

In [40]:
pokemon = get_all_pokemon()

In [42]:
with open('pokemon_urls.json', 'w') as f:
    json.dump(pokemon, f, indent=4)

### Getting more detailed data on each Pokemon

In [95]:
pokemon_fp = '/Users/dakotaleesmith/pyProjects/pokemon/pokemon_urls.json'
with open(pokemon_fp, 'r') as f:
     data = json.load(f)

In [101]:
session = requests.Session()

for pokemon in data:
    name = pokemon.get('name')
    url = pokemon.get('url')
    response = session.get(url)
    response.raise_for_status()
    response_json = response.json()
    fp = f'/Users/dakotaleesmith/pyProjects/pokemon/individual_pokemon/{name}.json'
    with open(fp, 'w') as f:
        json.dump(response_json, f, indent=4)

### Exploring individual Pokemon data

In [3]:
def get_a_pokemon(name: str) -> dict:
    fp = f'/Users/dakotaleesmith/pyProjects/pokemon/individual_pokemon/{name}.json'
    with open(fp, 'r') as f:
        return json.load(f)

In [4]:
name = 'charmander'
charmander = get_a_pokemon(name)
print(list(charmander.keys()), '\n')

dimension = 'species'
data = charmander.get(dimension)
print(data,'\n')

['abilities', 'base_experience', 'forms', 'game_indices', 'height', 'held_items', 'id', 'is_default', 'location_area_encounters', 'moves', 'name', 'order', 'past_types', 'species', 'sprites', 'stats', 'types', 'weight'] 

{'name': 'charmander', 'url': 'https://pokeapi.co/api/v2/pokemon-species/4/'} 



### Creating curated dataset

#### Raw pokedata

In [23]:
pokemon_dir = '/Users/dakotaleesmith/pyProjects/pokemon/individual_pokemon'
pokemon_files = os.listdir(pokemon_dir)

pokedata = []
for pokemon in pokemon_files:
    with open(f'{pokemon_dir}/{pokemon}', 'r') as f:
        pokedict = json.load(f)
    name = pokedict.get('name')
    types = pokedict.get('types')
    stats = pokedict.get('stats')
    species = pokedict.get('species')
    abilities = pokedict.get('abilities')
    data = {
        'name':name,
        'types':types,
        'stats':stats,
        'species':species,
        'abilities':abilities
    }
    pokedata.append(data)

In [24]:
pokedata[0]

{
    'name': 'latias-mega',
    'types': [
        {'slot': 1, 'type': {'name': 'dragon', 'url': 'https://pokeapi.co/api/v2/type/16/'}},
        {'slot': 2, 'type': {'name': 'psychic', 'url': 'https://pokeapi.co/api/v2/type/14/'}}
    ],
    'stats': [
        {
            'base_stat': 80,
            'effort': 0,
            'stat': {'name': 'hp', 'url': 'https://pokeapi.co/api/v2/stat/1/'}
        },
        {
            'base_stat': 100,
            'effort': 0,
            'stat': {'name': 'attack', 'url': 'https://pokeapi.co/api/v2/stat/2/'}
        },
        {
            'base_stat': 120,
            'effort': 0,
            'stat': {'name': 'defense', 'url': 'https://pokeapi.co/api/v2/stat/3/'}
        },
        {
            'base_stat': 140,
            'effort': 0,
            'stat': {'name': 'special-attack', 'url': 'https://pokeapi.co/api/v2/stat/4/'}
        },
        {
            'base_stat': 150,
            'effort': 3,
            'stat': {'name': 'special-defense', 'url': 'https://pokeapi.co/api/v2/stat/5/'}
        },
        {
            'base_stat': 110,
            'effort': 0,
            'stat': {'name': 'speed', 'url': 'https://pokeapi.co/api/v2/stat/6/'}
        }
    ],
    'species': {'name': 'latias', 'url': 'https://pokeapi.co/api/v2/pokemon-species/380/'},
    'abilities': [
        {
            'ability': {'name': 'levitate', 'url': 'https://pokeapi.co/api/v2/ability/26/'},
            'is_hidden': False,
            'slot': 1
        }
    ]
}

#### Formatting raw pokedata

In [29]:
class PokemonDataParser:
    def __init__(self, pokemon: dict):
        self.pokemon = pokemon

    def get_type(self):
        return [i.get('type').get('name') for i in self.pokemon.get('types')]
    
    def get_stats(self):
        return [{stat.get('stat').get('name'): stat.get('base_stat')} for stat in self.pokemon.get('stats')]
    
    def get_abilities(self):
        abilities_json_fp = '/Users/dakotaleesmith/pyProjects/pokemon/abilities.json'
        with open(abilities_json_fp, 'r') as f:
            data = json.load(f)
        return [data.get(ability.get('ability').get('name')) for ability in self.pokemon.get('abilities')]
        # abilities = []
        # for i in self.pokemon.get('abilities'):
        #     name = i.get('ability').get('name')
        #     ability = {'name':name, 'is_hidden':i.get('is_hidden')}
        #     abilities.append(ability)
        # return abilities
    
    def get_abilities_detail(self):
        abilities = self.get_abilities()
        abilities_json_fp = '/Users/dakotaleesmith/pyProjects/pokemon/abilities.json'
        with open(abilities_json_fp, 'r') as f:
            data = json.load(f)
        for ability in abilities:
            # name = ability.get('name')
            effect_short= ability.get('effect_short')
            # effect_short = data.get(name).get('effect_short')
            ability.update({'effect_short':effect_short})
        return abilities

            
def format_pokedata(pokemon_list: list):
    data = []
    for pokemon in pokemon_list:
        pokedata_parser = PokemonDataParser(pokemon)
        schema = {
            'name':pokemon.get('name'),
            'types':pokedata_parser.get_type(),
            'stats':pokedata_parser.get_stats(),
            'abilities':pokedata_parser.get_abilities(),
            'evolutions_url':pokemon.get('species').get('url')
        }
        data.append(schema)
    return data

format_pokedata(pokedata[0])

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/f0/jbpk1lns7ql16c186r_ktfwm0000gn/T/ipykernel_50858/4287790235.py:50 in <module>    │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/f0/jbpk1lns7ql16c186r_ktfwm0000gn/T/ipykernel_50858/4287790235.py'                 │
│                                                                                                  │
│ /var/folders/f0/jbpk1lns7ql16c186r_ktfwm0000gn/T/ipykernel_50858/4287790235.py:41 in             │
│ format_pokedata                                                                                  │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/f0/jbpk1lns7ql16c186r_ktfwm0000gn/T/ipykernel_50858/4287790235.py'                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'str' object has no attribute 'get'

In [83]:
formatted_pokedata = format_pokedata(pokedata)
formatted_pokedata_fp = '/Users/dakotaleesmith/pyProjects/pokemon/formatted_pokedata.json'
with open(formatted_pokedata_fp, 'w') as f:
    json.dump(formatted_pokedata, f, indent=4)

#### Exploring abilities endpoint/response

In [73]:
res = requests.get('https://pokeapi.co/api/v2/ability/libero')
res.raise_for_status()
libero_data = res.json()
if not libero_data.get('effect_entries'):
    print([i.get('flavor_text') for i in libero_data.get('flavor_text_entries') if i.get('language').get('name') == 'en'])

['Changes the Pokémon’s type to the type of the\nmove it’s about to use.']


In [24]:
sample_ability = get_pokemon_response(formatted_pokedata[0].get('abilities')[0].get('url'))
sample_ability.keys()

dict_keys(['effect_changes', 'effect_entries', 'flavor_text_entries', 'generation', 'id', 'is_main_series', 'name', 'names', 'pokemon'])

In [33]:
sample_ability.get('effect_entries')

[
    {
        'effect': 'Ein Pokémon mit dieser Fähigkeit ist immun gegen ground Attacken, spikes, toxic spikes und arena trap.\n\nDiese Fähigkeit wird mit gravity oder ingrain deaktiviert oder während eine iron ball getragen wird. roost deaktiviert diese Fähigkeit nicht.',
        'language': {'name': 'de', 'url': 'https://pokeapi.co/api/v2/language/6/'},
        'short_effect': 'Verhindert ground Attacken'
    },
    {
        'effect': 'This Pokémon is immune to ground-type moves, spikes, toxic spikes, and arena trap.\n\nThis ability is disabled during gravity or ingrain, or while holding an iron ball.  This ability is not disabled during roost.',
        'language': {'name': 'en', 'url': 'https://pokeapi.co/api/v2/language/9/'},
        'short_effect': 'Evades ground moves.'
    }
]

In [28]:
[i.get('effect') for i in sample_ability.get('effect_entries') if i.get('language').get('name') == 'en']

[
    'This Pokémon is immune to ground-type moves, spikes, toxic spikes, and arena trap.\n\nThis ability is disabled during gravity or ingrain, or while holding an iron ball.  This ability is not disabled during roost.'
]

#### Saving abilities data

In [32]:
urls = []
for pokemon in formatted_pokedata:
    abilities = pokemon.get('abilities')
    urls.extend([ability.get('url') for ability in abilities])

urls = set(urls)
len(urls)

298

In [79]:
with requests.Session() as session:
    abilities = []
    for url in urls:
        response = session.get(url)
        response.raise_for_status()
        data = response.json()
        if data.get('effect_entries'):
            ability = [
                {
                    'name':data.get('name'),
                    'effect':i.get('effect'),
                    'effect_short':i.get('short_effect')
                }
                for i in data.get('effect_entries')
                if i.get('language').get('name') == 'en'
            ]
        elif data.get('flavor_text_entries'):
            ability = [
                {
                'name':data.get('name'),
                'effect':data.get('flavor_text'),
                'effect_short':data.get('flavor_text')
                }
                for i in data.get('flavor_text_entries')
                if i.get('language').get('name') == 'en'
            ]
        else:
            ability = [
                {
                'name':data.get('name'),
                'effect':'Data not available.',
                'effect_short':'Data not available.'
                }
            ]
        abilities.extend(ability)

In [80]:
abilities_formatted = {i.get('name'): {'effect':i.get('effect'), 'effect_short':i.get('effect_short')} for i in abilities}
len(abilities_formatted)

298

In [81]:
with open('abilities.json', 'w') as f:
    json.dump(abilities_formatted, f, indent=4)

#### Exploring evolutions endpoint/response

In [91]:
sample_species_data = get_pokemon_response(formatted_pokedata[3].get('evolutions_url'))
sample_evolutions_data = get_pokemon_response(sample_species_data.get('evolution_chain').get('url'))

list(sample_evolutions_data.keys())

['baby_trigger_item', 'chain', 'id']

In [92]:
sample_evolutions_data.get('chain')

{
    'evolution_details': [],
    'evolves_to': [
        {
            'evolution_details': [
                {
                    'gender': None,
                    'held_item': None,
                    'item': None,
                    'known_move': None,
                    'known_move_type': None,
                    'location': None,
                    'min_affection': None,
                    'min_beauty': None,
                    'min_happiness': None,
                    'min_level': 28,
                    'needs_overworld_rain': False,
                    'party_species': None,
                    'party_type': None,
                    'relative_physical_stats': None,
                    'time_of_day': '',
                    'trade_species': None,
                    'trigger': {
                        'name': 'level-up',
                        'url': 'https://pokeapi.co/api/v2/evolution-trigger/1/'
                    },
                    'turn_upside_down': False
                },
                {
                    'gender': None,
                    'held_item': None,
                    'item': None,
                    'known_move': None,
                    'known_move_type': None,
                    'location': None,
                    'min_affection': None,
                    'min_beauty': None,
                    'min_happiness': None,
                    'min_level': 28,
                    'needs_overworld_rain': False,
                    'party_species': None,
                    'party_type': None,
                    'relative_physical_stats': None,
                    'time_of_day': 'night',
                    'trade_species': None,
                    'trigger': {
                        'name': 'level-up',
                        'url': 'https://pokeapi.co/api/v2/evolution-trigger/1/'
                    },
                    'turn_upside_down': False
                }
            ],
            'evolves_to': [],
            'is_baby': False,
            'species': {'name': 'marowak', 'url': 'https://pokeapi.co/api/v2/pokemon-species/105/'}
        }
    ],
    'is_baby': False,
    'species': {'name': 'cubone', 'url': 'https://pokeapi.co/api/v2/pokemon-species/104/'}
}

#### Getting type effectiveness

In [36]:
fp = '/Users/dakotaleesmith/pyProjects/pokemon/formatted_pokedata.json'
with open(fp, 'r') as f:
    formatted_pokedata = json.load(f)

In [3]:
type_urls = []
for pokemon in formatted_pokedata:
    for type in pokemon.get('types'):
        url = type.get('url')
        if url not in type_urls:
            type_urls.append(url)

In [4]:
types = []

with requests.Session() as session:
    for url in type_urls:
        response = session.get(url)
        response.raise_for_status()
        data = response.json()
        types.append(data)

with open('types.json', 'w') as f:
    json.dump(types, f, indent=4)

In [21]:
types_parsed = []
for type in types:
    type_name = type.get('name')
    damage_relations = type.get('damage_relations')
    for k, v in damage_relations.items():
        if v:
            type_names = [i.get('name') for i in v]
            relation = {'type': type_name, k: type_names}
            types_parsed.append(relation)

with open('types_parsed.json', 'w') as f:
    json.dump(types_parsed, f, indent=4)

In [37]:
for pokemon in formatted_pokedata:
    pokemon['types'] = [type.get('name')for type in pokemon.get('types')]

In [74]:
with open('types_parsed2.json', 'r') as f:
    types = json.load(f)

types

{
    'electric': [
        {'double_damage_from': ['ground']},
        {'double_damage_to': ['flying', 'water']},
        {'half_damage_from': ['flying', 'steel', 'electric']},
        {'half_damage_to': ['grass', 'electric', 'dragon']},
        {'no_damage_to': ['ground']}
    ],
    'steel': [
        {'double_damage_from': ['fighting', 'ground', 'fire']},
        {'double_damage_to': ['rock', 'ice', 'fairy']},
        {
            'half_damage_from': [
                'normal',
                'flying',
                'rock',
                'bug',
                'steel',
                'grass',
                'psychic',
                'ice',
                'dragon',
                'fairy'
            ]
        },
        {'half_damage_to': ['steel', 'fire', 'water', 'electric']},
        {'no_damage_from': ['poison']}
    ],
    'ice': [
        {'double_damage_from': ['fighting', 'rock', 'steel', 'fire']},
        {'double_damage_to': ['flying', 'ground', 'grass', 'dragon']},
        {'half_damage_from': ['ice']},
        {'half_damage_to': ['steel', 'fire', 'water', 'ice']}
    ],
    'rock': [
        {'double_damage_from': ['fighting', 'ground', 'steel', 'water', 'grass']},
        {'double_damage_to': ['flying', 'bug', 'fire', 'ice']},
        {'half_damage_from': ['normal', 'flying', 'poison', 'fire']},
        {'half_damage_to': ['fighting', 'ground', 'steel']}
    ],
    'normal': [
        {'double_damage_from': ['fighting']},
        {'half_damage_to': ['rock', 'steel']},
        {'no_damage_from': ['ghost']},
        {'no_damage_to': ['ghost']}
    ],
    'dragon': [
        {'double_damage_from': ['ice', 'dragon', 'fairy']},
        {'double_damage_to': ['dragon']},
        {'half_damage_from': ['fire', 'water', 'grass', 'electric']},
        {'half_damage_to': ['steel']},
        {'no_damage_to': ['fairy']}
    ],
    'fire': [
        {'double_damage_from': ['ground', 'rock', 'water']},
        {'double_damage_to': ['bug', 'steel', 'grass', 'ice']},
        {'half_damage_from': ['bug', 'steel', 'fire', 'grass', 'ice', 'fairy']},
        {'half_damage_to': ['rock', 'fire', 'water', 'dragon']}
    ],
    'flying': [
        {'double_damage_from': ['rock', 'electric', 'ice']},
        {'double_damage_to': ['fighting', 'bug', 'grass']},
        {'half_damage_from': ['fighting', 'bug', 'grass']},
        {'half_damage_to': ['rock', 'steel', 'electric']},
        {'no_damage_from': ['ground']}
    ],
    'bug': [
        {'double_damage_from': ['flying', 'rock', 'fire']},
        {'double_damage_to': ['grass', 'psychic', 'dark']},
        {'half_damage_from': ['fighting', 'ground', 'grass']},
        {'half_damage_to': ['fighting', 'flying', 'poison', 'ghost', 'steel', 'fire', 'fairy']}
    ],
    'dark': [
        {'double_damage_from': ['fighting', 'bug', 'fairy']},
        {'double_damage_to': ['ghost', 'psychic']},
        {'half_damage_from': ['ghost', 'dark']},
        {'half_damage_to': ['fighting', 'dark', 'fairy']},
        {'no_damage_from': ['psychic']}
    ],
    'grass': [
        {'double_damage_from': ['flying', 'poison', 'bug', 'fire', 'ice']},
        {'double_damage_to': ['ground', 'rock', 'water']},
        {'half_damage_from': ['ground', 'water', 'grass', 'electric']},
        {'half_damage_to': ['flying', 'poison', 'bug', 'steel', 'fire', 'grass', 'dragon']}
    ],
    'ground': [
        {'double_damage_from': ['water', 'grass', 'ice']},
        {'double_damage_to': ['poison', 'rock', 'steel', 'fire', 'electric']},
        {'half_damage_from': ['poison', 'rock']},
        {'half_damage_to': ['bug', 'grass']},
        {'no_damage_from': ['electric']},
        {'no_damage_to': ['flying']}
    ],
    'water': [
        {'double_damage_from': ['grass', 'electric']},
        {'double_damage_to': ['ground', 'rock', 'fire']},
        {'half_damage_from': ['steel', 'fire', 'water', 'ice']},
        {'half_damage_to': ['water', 'grass', 'dragon']}
    ],
    'fairy': [
        {'double_damage_

In [75]:
for k, v in types.items():
    types[k] = {list(i.keys())[0]:list(i.values())[0] for i in v}

types

{
    'electric': {
        'double_damage_from': ['ground'],
        'double_damage_to': ['flying', 'water'],
        'half_damage_from': ['flying', 'steel', 'electric'],
        'half_damage_to': ['grass', 'electric', 'dragon'],
        'no_damage_to': ['ground']
    },
    'steel': {
        'double_damage_from': ['fighting', 'ground', 'fire'],
        'double_damage_to': ['rock', 'ice', 'fairy'],
        'half_damage_from': [
            'normal',
            'flying',
            'rock',
            'bug',
            'steel',
            'grass',
            'psychic',
            'ice',
            'dragon',
            'fairy'
        ],
        'half_damage_to': ['steel', 'fire', 'water', 'electric'],
        'no_damage_from': ['poison']
    },
    'ice': {
        'double_damage_from': ['fighting', 'rock', 'steel', 'fire'],
        'double_damage_to': ['flying', 'ground', 'grass', 'dragon'],
        'half_damage_from': ['ice'],
        'half_damage_to': ['steel', 'fire', 'water', 'ice']
    },
    'rock': {
        'double_damage_from': ['fighting', 'ground', 'steel', 'water', 'grass'],
        'double_damage_to': ['flying', 'bug', 'fire', 'ice'],
        'half_damage_from': ['normal', 'flying', 'poison', 'fire'],
        'half_damage_to': ['fighting', 'ground', 'steel']
    },
    'normal': {
        'double_damage_from': ['fighting'],
        'half_damage_to': ['rock', 'steel'],
        'no_damage_from': ['ghost'],
        'no_damage_to': ['ghost']
    },
    'dragon': {
        'double_damage_from': ['ice', 'dragon', 'fairy'],
        'double_damage_to': ['dragon'],
        'half_damage_from': ['fire', 'water', 'grass', 'electric'],
        'half_damage_to': ['steel'],
        'no_damage_to': ['fairy']
    },
    'fire': {
        'double_damage_from': ['ground', 'rock', 'water'],
        'double_damage_to': ['bug', 'steel', 'grass', 'ice'],
        'half_damage_from': ['bug', 'steel', 'fire', 'grass', 'ice', 'fairy'],
        'half_damage_to': ['rock', 'fire', 'water', 'dragon']
    },
    'flying': {
        'double_damage_from': ['rock', 'electric', 'ice'],
        'double_damage_to': ['fighting', 'bug', 'grass'],
        'half_damage_from': ['fighting', 'bug', 'grass'],
        'half_damage_to': ['rock', 'steel', 'electric'],
        'no_damage_from': ['ground']
    },
    'bug': {
        'double_damage_from': ['flying', 'rock', 'fire'],
        'double_damage_to': ['grass', 'psychic', 'dark'],
        'half_damage_from': ['fighting', 'ground', 'grass'],
        'half_damage_to': ['fighting', 'flying', 'poison', 'ghost', 'steel', 'fire', 'fairy']
    },
    'dark': {
        'double_damage_from': ['fighting', 'bug', 'fairy'],
        'double_damage_to': ['ghost', 'psychic'],
        'half_damage_from': ['ghost', 'dark'],
        'half_damage_to': ['fighting', 'dark', 'fairy'],
        'no_damage_from': ['psychic']
    },
    'grass': {
        'double_damage_from': ['flying', 'poison', 'bug', 'fire', 'ice'],
        'double_damage_to': ['ground', 'rock', 'water'],
        'half_damage_from': ['ground', 'water', 'grass', 'electric'],
        'half_damage_to': ['flying', 'poison', 'bug', 'steel', 'fire', 'grass', 'dragon']
    },
    'ground': {
        'double_damage_from': ['water', 'grass', 'ice'],
        'double_damage_to': ['poison', 'rock', 'steel', 'fire', 'electric'],
        'half_damage_from': ['poison', 'rock'],
        'half_damage_to': ['bug', 'grass'],
        'no_damage_from': ['electric'],
        'no_damage_to': ['flying']
    },
    'water': {
        'double_damage_from': ['grass', 'electric'],
        'double_damage_to': ['ground', 'rock', 'fire'],
        'half_damage_from': ['steel', 'fire', 'water', 'ice'],
        'half_damage_to': ['water', 'grass', 'dragon']
    },
    'fairy': {
        'double_damage_from': ['poison', 'steel'],
        'double_damage_to': ['fighting', 'dragon', 'dark'],
        'half_damage_from': ['fighting', 'bug', 'dark'],
        'half_damage_to': ['poison', 'st

In [78]:
with open('types_parsed2.json', 'w') as f:
    json.dump(types, f, indent=4)